In [2]:
#search for change in dice loss for single sample
import pandas as pd
import numpy as np
import os
from monai.metrics import compute_dice
import glob
import torch


/home/vscode/.local/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
dvolumeT = pd.read_csv('/workspaces/data/MegaGen/logs/SCORE/CSVS/id_test_volume.csv')
dvolTS = dvolumeT[dvolumeT['volume'] <3000]
dvolTS['id'] = dvolTS['id'].apply(lambda x: f'{int(x):04d}')

/tmp/ipykernel_38845/821571249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dvolTS['id'] = dvolTS['id'].apply(lambda x: f'{int(x):04d}')


In [19]:
root = '/workspaces/data/brain_meningioma'
def getScoreS(df_slice):
    dScore = pd.DataFrame() # columns=['maskTfile', 'dscore'])
    for id in df_slice['id']:
        maskTfile = os.path.join('npy', f'BraTS-MEN-RT-{id}-1_slice_*_mask.npy')
        # probTfile = os.path.join('pt', f'BraTS-MEN-RT-{id}-1_slice_*_mask.pt')
        maskFiles = glob.glob(os.path.join(root, 'slice/s_test', maskTfile))
        for maskFile in maskFiles:
            probFile0 = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile0 = os.path.join(root, 'oProb/unet1s/s_test', probFile0)
            y_pred0 = torch.load(probFile0,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
            
            probFile1 = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile1 = os.path.join(root, 'oProb/unetpp0D/s_test', probFile1)
            y_pred1 = torch.load(probFile1,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
            
            y = torch.from_numpy(np.load(maskFile)).unsqueeze(0).unsqueeze(0).cuda(0)
            d0= compute_dice(y_pred0, y).item()
            d1= compute_dice(y_pred1, y).item()
            dScore = pd.concat([dScore, pd.DataFrame
                                ([[os.path.relpath(maskFile,os.path.join(root,'slice/s_test')), d0, d1]], 
                                    columns=['maskTfile', 'd0','d1'])])
    return dScore
        
    # probIFiles = glob.glob(f'/workspaces/data/brain_meningioma/oProb/unet1s/s_test/pt/BraTS-MEN-RT-{id}-2_slice_*.pt')


In [38]:
maskTfile

'npy/BraTS-MEN-RT-0109-1_slice_*_mask.npy'

In [ ]:
dScore = getScoreS(dvolTS)
dScore['dadd'] = dScore['d1'] - dScore['d0']

In [49]:
dScore.sort_values(by='dadd', ascending=False).head(20)

,maskTfile,d0,d1,dadd
0,npy/BraTS-MEN-RT-0264-1_slice_062_mask.npy,0.000000,0.800000,0.800000
0,npy/BraTS-MEN-RT-0053-1_slice_124_mask.npy,0.000000,0.742857,0.742857
0,npy/BraTS-MEN-RT-0165-1_slice_068_mask.npy,0.000000,0.742358,0.742358
0,npy/BraTS-MEN-RT-0264-1_slice_063_mask.npy,0.102041,0.772727,0.670686
0,npy/BraTS-MEN-RT-0165-1_slice_067_mask.npy,0.190045,0.853755,0.663710
0,npy/BraTS-MEN-RT-0053-1_slice_123_mask.npy,0.299065,0.862069,0.563004
0,npy/BraTS-MEN-RT-0264-1_slice_064_mask.npy,0.000000,0.548387,0.548387
0,npy/BraTS-MEN-RT-0053-1_slice_128_mask.npy,0.343750,0.881356,0.537606
0,npy/BraTS-MEN-RT-0092-1_slice_066_mask.npy,0.000000,0.535211,0.535211
0,npy/BraTS-MEN-RT-0320-1_slice_168_mask.npy,0.382114,0.850394,0.468280


In [49]:
dLobe = pd.read_csv('/workspaces/data/MegaGen/logs/SCORE/CSVS/id_test_lobe.csv',index_col=0)
dLobeT = dLobe[(dLobe['lobe_overlap'] == 'Temporal') | (dLobe['lobe_overlap'] == 'Frontal')]
dLobeT['id'] = dLobeT['id'].apply(lambda x: f'{int(x):04d}')
#dScoreLT = getScoreS(dLobeT)
#dScoreLT['dadd'] = dScoreLT['d1'] - dScoreLT['d0']

/tmp/ipykernel_38845/3928166188.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dLobeT['id'] = dLobeT['id'].apply(lambda x: f'{int(x):04d}')


In [51]:
dLobeT[dLobeT['id']=='0385']

,id,lobe_overlap
35,0385,Frontal


In [21]:
dScoreLT.sort_values(by='dadd', ascending=False).head(20)

,maskTfile,d0,d1,dadd
0,npy/BraTS-MEN-RT-0264-1_slice_062_mask.npy,0.000000,0.800000,0.800000
0,npy/BraTS-MEN-RT-0165-1_slice_068_mask.npy,0.000000,0.742358,0.742358
0,npy/BraTS-MEN-RT-0264-1_slice_063_mask.npy,0.102041,0.772727,0.670686
0,npy/BraTS-MEN-RT-0165-1_slice_067_mask.npy,0.190045,0.853755,0.663710
0,npy/BraTS-MEN-RT-0264-1_slice_064_mask.npy,0.000000,0.548387,0.548387
0,npy/BraTS-MEN-RT-0092-1_slice_066_mask.npy,0.000000,0.535211,0.535211
0,npy/BraTS-MEN-RT-0092-1_slice_049_mask.npy,0.441441,0.861111,0.419670
0,npy/BraTS-MEN-RT-0092-1_slice_065_mask.npy,0.000000,0.340000,0.340000
0,npy/BraTS-MEN-RT-0610-1_slice_095_mask.npy,0.473856,0.778325,0.304469
0,npy/BraTS-MEN-RT-0092-1_slice_063_mask.npy,0.000000,0.304094,0.304094


In [11]:
#trying 3-dice
root = '/workspaces/data/brain_meningioma'
dScore = pd.DataFrame() # columns=['maskTfile', 'dscore'])
for id in dvolTS['id']:
    maskTfile = os.path.join('npy', f'BraTS-MEN-RT-{id}-1_slice_*_mask.npy')
    # probTfile = os.path.join('pt', f'BraTS-MEN-RT-{id}-1_slice_*_mask.pt')
    maskFiles = glob.glob(os.path.join(root, 'slice/s_test', maskTfile))
    for maskFile in maskFiles:
        y_preds = []
        for model in ['unet1s', 'unetpp0', 'unetpp0D']:
            probFile = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile = os.path.join(root, f'oProb/{model}/s_test', probFile)
            y_pred = torch.load(probFile,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
            y_preds.append(y_pred)
        
        y = torch.from_numpy(np.load(maskFile)).unsqueeze(0).unsqueeze(0).cuda(0)
        dices = [compute_dice(y_pred, y).item() for y_pred in y_preds]
        dScore = pd.concat([dScore, pd.DataFrame
                            ([[id,os.path.relpath(maskFile,os.path.join(root,'slice/s_test')), dices]], 
                                columns=['id','maskTfile', 'dices'])])
    
    # probIFiles = glob.glob(f'/workspaces/data/brain_meningioma/oProb/unet1s/s_test/pt/BraTS-MEN-RT-{id}-2_slice_*.pt')

In [ ]:
dScore['maskTitle']

,maskTfile,dices
0,npy/BraTS-MEN-RT-0087-1_slice_133_mask.npy,"[0.0, 0.0, 0.0]"
0,npy/BraTS-MEN-RT-0087-1_slice_134_mask.npy,"[0.75, 0.0, 0.7407407164573669]"
0,npy/BraTS-MEN-RT-0087-1_slice_135_mask.npy,"[0.4047619104385376, 0.0, 0.8421052694320679]"
0,npy/BraTS-MEN-RT-0087-1_slice_136_mask.npy,"[0.4307692348957062, 0.0, 0.8484848737716675]"
0,npy/BraTS-MEN-RT-0087-1_slice_137_mask.npy,"[0.7222222089767456, 0.43478259444236755, 0.8125]"
...,...,...
0,npy/BraTS-MEN-RT-0109-1_slice_058_mask.npy,"[0.7864077687263489, 0.8125, 0.7844827771186829]"
0,npy/BraTS-MEN-RT-0109-1_slice_059_mask.npy,"[0.8888888955116272, 0.8648648858070374, 0.837..."
0,npy/BraTS-MEN-RT-0109-1_slice_060_mask.npy,"[0.8487805128097534, 0.7837837934494019, 0.792..."
0,npy/BraTS-MEN-RT-0109-1_slice_061_mask.npy,"[0.27067670226097107, 0.0, 0.3636363744735718]"


In [12]:
dScore['dadd'] = dScore['dices'].apply(lambda x:x[2])-dScore['dices'].apply(lambda x:x[0])

In [ ]:
dScoreH = dScore.sort_values(by='dadd', ascending=False).head(20)# .to_numpy()
dScoreH[dScoreH['id'].isin(['0165','0320','0053','0256','0385'])].to_numpy()

array([['0053', 'npy/BraTS-MEN-RT-0053-1_slice_124_mask.npy',
        list([0.0, 0.42553192377090454, 0.7428571581840515]),
        0.7428571581840515],
       ['0165', 'npy/BraTS-MEN-RT-0165-1_slice_068_mask.npy',
        list([0.0, 0.5793103575706482, 0.7423580884933472]),
        0.7423580884933472],
       ['0165', 'npy/BraTS-MEN-RT-0165-1_slice_067_mask.npy',
        list([0.19004525244235992, 0.6805970072746277, 0.8537549376487732]),
        0.6637096852064133],
       ['0053', 'npy/BraTS-MEN-RT-0053-1_slice_123_mask.npy',
        list([0.29906541109085083, 0.5316455960273743, 0.8620689511299133]),
        0.5630035400390625],
       ['0053', 'npy/BraTS-MEN-RT-0053-1_slice_128_mask.npy',
        list([0.34375, 0.5789473652839661, 0.8813559412956238]),
        0.5376059412956238],
       ['0320', 'npy/BraTS-MEN-RT-0320-1_slice_168_mask.npy',
        list([0.38211381435394287, 0.39382240176200867, 0.8503937125205994]),
        0.4682798981666565]], dtype=object)

In [57]:
dScoreLobe = pd.DataFrame() # columns=['maskTfile', 'dscore'])
for id in dLobeT['id']:
    maskTfile = os.path.join('npy', f'BraTS-MEN-RT-{id}-1_slice_*_mask.npy')
    # probTfile = os.path.join('pt', f'BraTS-MEN-RT-{id}-1_slice_*_mask.pt')
    maskFiles = glob.glob(os.path.join(root, 'slice/s_test', maskTfile))
    for maskFile in maskFiles:
        y_preds = []
        for model in ['unet1s', 'unetpp0', 'unetpp0D']:
            probFile = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile = os.path.join(root, f'oProb/{model}/s_test', probFile)
            y_pred = torch.load(probFile,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
            y_preds.append(y_pred)
        
        y = torch.from_numpy(np.load(maskFile)).unsqueeze(0).unsqueeze(0).cuda(0)
        dices = [compute_dice(y_pred, y).item() for y_pred in y_preds]
        dScoreLobe = pd.concat([dScoreLobe, pd.DataFrame
                            ([[id,os.path.relpath(maskFile,os.path.join(root,'slice/s_test')), dices]], 
                                columns=['id','maskTfile', 'dices'])])
dScoreLobe['dadd'] = dScoreLobe['dices'].apply(lambda x:x[2])-dScoreLobe['dices'].apply(lambda x:x[0])
dScoreLobe.sort_values(by='dadd', ascending=False).head(10).to_numpy()

array([['0385', 'npy/BraTS-MEN-RT-0385-1_slice_095_mask.npy',
        list([0.0031645570416003466, 0.7820636630058289, 0.9389439225196838]),
        0.9357793654780835],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_092_mask.npy',
        list([0.008486563339829445, 0.7938730716705322, 0.9265917539596558]),
        0.9181051906198263],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_088_mask.npy',
        list([0.0, 0.16252519190311432, 0.8984375]), 0.8984375],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_091_mask.npy',
        list([0.07215792685747147, 0.7962568998336792, 0.9166666865348816]),
        0.8445087596774101],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_090_mask.npy',
        list([0.087013840675354, 0.5977822542190552, 0.9227913618087769]),
        0.8357775211334229],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_093_mask.npy',
        list([0.1187279149889946, 0.7593632936477661, 0.9399075508117676]),
        0.821179635822773],
       ['0264', 'npy/BraTS-MEN-RT-

In [56]:
dScoreLobe

,id,maskTfile,dices,dadd
0,0610,npy/BraTS-MEN-RT-0610-1_slice_101_mask.npy,"[0.8260869383811951, 0.8172042965888977, 0.808...",-0.017576


In [60]:
dScoreLH = dScoreLobe.sort_values(by='dadd', ascending=False).head(20)# .to_numpy()
dScoreLH[dScoreLH['id'].isin(['0385'])].to_numpy()

array([['0385', 'npy/BraTS-MEN-RT-0385-1_slice_095_mask.npy',
        list([0.0031645570416003466, 0.7820636630058289, 0.9389439225196838]),
        0.9357793654780835],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_092_mask.npy',
        list([0.008486563339829445, 0.7938730716705322, 0.9265917539596558]),
        0.9181051906198263],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_088_mask.npy',
        list([0.0, 0.16252519190311432, 0.8984375]), 0.8984375],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_091_mask.npy',
        list([0.07215792685747147, 0.7962568998336792, 0.9166666865348816]),
        0.8445087596774101],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_090_mask.npy',
        list([0.087013840675354, 0.5977822542190552, 0.9227913618087769]),
        0.8357775211334229],
       ['0385', 'npy/BraTS-MEN-RT-0385-1_slice_093_mask.npy',
        list([0.1187279149889946, 0.7593632936477661, 0.9399075508117676]),
        0.821179635822773],
       ['0385', 'npy/BraTS-MEN-RT-

In [61]:
dScoreLH[dScoreLH['id']=='0385']

,id,maskTfile,dices,dadd
0,0385,npy/BraTS-MEN-RT-0385-1_slice_095_mask.npy,"[0.0031645570416003466, 0.7820636630058289, 0....",0.935779
0,0385,npy/BraTS-MEN-RT-0385-1_slice_092_mask.npy,"[0.008486563339829445, 0.7938730716705322, 0.9...",0.918105
0,0385,npy/BraTS-MEN-RT-0385-1_slice_088_mask.npy,"[0.0, 0.16252519190311432, 0.8984375]",0.898438
0,0385,npy/BraTS-MEN-RT-0385-1_slice_091_mask.npy,"[0.07215792685747147, 0.7962568998336792, 0.91...",0.844509
0,0385,npy/BraTS-MEN-RT-0385-1_slice_090_mask.npy,"[0.087013840675354, 0.5977822542190552, 0.9227...",0.835778
0,0385,npy/BraTS-MEN-RT-0385-1_slice_093_mask.npy,"[0.1187279149889946, 0.7593632936477661, 0.939...",0.821180
0,0385,npy/BraTS-MEN-RT-0385-1_slice_089_mask.npy,"[0.17191600799560547, 0.1937207728624344, 0.89...",0.725184
0,0385,npy/BraTS-MEN-RT-0385-1_slice_086_mask.npy,"[0.0, 0.0, 0.6695699691772461]",0.669570
0,0385,npy/BraTS-MEN-RT-0385-1_slice_094_mask.npy,"[0.2745901644229889, 0.809769868850708, 0.9367...",0.662201
0,0385,npy/BraTS-MEN-RT-0385-1_slice_085_mask.npy,"[0.009615384973585606, 0.050673000514507294, 0...",0.539334


In [43]:
#3D dice

root = '/workspaces/data/brain_meningioma'
dScore = pd.DataFrame() # columns=['maskTfile', 'dscore'])
for id in dvolTS['id']:
    maskTfile = os.path.join('npy', f'BraTS-MEN-RT-{id}-1_slice_*_mask.npy')
    # probTfile = os.path.join('pt', f'BraTS-MEN-RT-{id}-1_slice_*_mask.pt')
    maskFiles = sorted(glob.glob(os.path.join(root, 'slice/s_test', maskTfile)))
    for model in ['unet1s', 'unetpp0', 'unetpp0D']:
        y_predIDSs = []
        for maskFile in maskFiles:
            probFile = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile = os.path.join(root, f'oProb/{model}/s_test', probFile)
            y_pred = torch.load(probFile,weights_only=False)>0.5
            print('y_pred shape: ', y_pred.shape)
            y_predIDSs.append(y_pred)
            #torch.stack(y_predID)
            #rearrange(y_predID, )

        # y = torch.from_numpy(np.load(maskFile)).unsqueeze(0).unsqueeze(0).cuda(0)
        # dices = [compute_dice(y_pred, y).item() for y_pred in y_preds]
        # dScore = pd.concat([dScore, pd.DataFrame
        #                     ([[os.path.relpath(maskFile,os.path.join(root,'slice/s_test')), dices]], 
        #                         columns=['maskTfile', 'dices'])])
    

y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1,